In [1]:
%run testcong.py
%run XE7

### Read in (or compute) the has table of $a_p\pmod{7}$ for $30$ primes over $400000$.  Computing this takes about 45 minutes

In [2]:
try:
    hashtab7_30 = load('hashtab7_30')
except IOError:
    hashtab7_30 = make_hash(7,11,400000,30)
    save(hashtab7_30, 'hashtab7_30')

### For each set of size ${}>1$, use Kraus's criterion to prove that the curves really are $7$-congruent.  Note that  we only test $p<10^7$ even if Kraus needs more, so the cases where a warning is output should be run again (this was done once outside this notebook).  NB This takes a long time.

In [133]:
bad_pairs = []
for s in hashtab7_30.values():
     if len(s)>1:
         E1 = EllipticCurve(s[0])
         for r in s[1:]:
             E2 = EllipticCurve(r)
             res, info = test_cong(7,E1,E2, mumax=10^7)
             if not res:
                 report(res,info,7,s[0],r)
                 bad_pairs.append([s[0],r])


## The result of the previous cell is that there is just one bad pair:

In [3]:
#bad_pairs
bad_pairs = [['25921a1', '78400gw1']] # previous cell takes ages; this is the output

## We extract all the sets of size greater than 1:

In [4]:
isom_sets = [s for s in hashtab7_30.values() if len(s)>1]
len(isom_sets)

20138

## Separate out the reducible and irreducible sets:

In [5]:
isom_sets_red = []
isom_sets_irred = []
for s in isom_sets:
    isom_sets_irred.append(s) if test_irred(s) else isom_sets_red.append(s)
print("{} irreducible sets, {} reducible sets".format(len(isom_sets_irred),len(isom_sets_red)))

19883 irreducible sets, 255 reducible sets


## The long test found that 25921a is not actually 7-congruent to 78400gw1, so we remove it from that set of 4 to leave a set of 3

In [6]:
bad_s = [s for s in isom_sets_red if '25921a1' in s][0]
bad_s

['25921a1', '78400gw1', '336973z1', '336973ba1']

## Check that 3 of the 4 curves in this set really are congruent:

In [7]:
print("Testing curves in {} for congruence".format(bad_s))
E0 = EllipticCurve(bad_s[0])
#print("{}: {}".format(E0.label(), E0))
E1 = EllipticCurve(bad_s[1])
#print("{}: {}".format(E1.label(), E1))
E2 = EllipticCurve(bad_s[2])
#print("{}: {}".format(E2.label(), E2))
E3 = EllipticCurve(bad_s[3])
#print("{}: {}".format(E3.label(), E3))
verb=False
res, info = test_cong(7,E0,E1, mumax=10^7, verbose=verb)
assert not res
print("{} and {} are not congruent mod 7: {}".format(bad_s[0],bad_s[1],info))
assert test_cong(7,E0,E2, mumax=10^7, verbose=verb)
print("{} and {} are congruent mod 7".format(bad_s[0],bad_s[2]))
assert test_cong(7,E0,E3, mumax=10^7, verbose=verb)
print("{} and {} are congruent mod 7".format(bad_s[0],bad_s[3]))
#assert test_cong(7,E2,E3, mumax=10^7, verbose=verb)
#print("{} and {} are congruent mod 7".format(bad_s[2],bad_s[3]))

Testing curves in ['25921a1', '78400gw1', '336973z1', '336973ba1'] for congruence
Curves 49a1 and 78400gw1: testing ell up to 14837760 mod 7
25921a1 and 78400gw1 are not congruent mod 7: (29, 2, -2)
25921a1 and 336973z1 are congruent mod 7
25921a1 and 336973ba1 are congruent mod 7


## Remove 78400gw1 from this set, and replace this set in the list of congruent sets:

In [8]:
bad_s_ok = [bad_s[i] for i in 0,2,3]
bad_s_ok

['25921a1', '336973z1', '336973ba1']

In [9]:
#ind = isom_sets_red.index(bad_s)
ind = 168

In [10]:
isom_sets_red[ind]=bad_s_ok

In [11]:
assert not [s for s in isom_sets_red if '78400gw1' in s]

## Count the numbers of sets of irreducible and reducible congruent classes, and the siezes of these sets:

In [12]:
from collections import Counter
red_sizes = Counter()
irred_sizes = Counter()

In [13]:
for s in isom_sets_irred:
    irred_sizes[len(s)] += 1
irred_sizes

Counter({2: 18334, 3: 1297, 4: 246, 5: 6})

### So most irreducible sets have size 2 and the maximum size is 6

In [14]:
for s in isom_sets_red:
    red_sizes[len(s)] += 1
red_sizes  

Counter({2: 86, 3: 67, 4: 31, 5: 26, 7: 7, 10: 6, 6: 5, 8: 5, 9: 4, 19: 4, 11: 3, 16: 2, 18: 2, 32: 1, 35: 1, 76: 1, 13: 1, 14: 1, 45: 1, 36: 1})

### Reducible sets have various sizes up to 76

## Next we go through all the irreducible sets, taking all the curves in each isogeny class and partitioning them into 1 or 2 subsets such that curves in the same subset are symplectically isomorphic, while curves in different subsets are antisymplectically isomorphic.

### The code which does the work is in testcong.py

 all_c is a function defined in testcong.py which, given the label of one curve E in a 7-irreducible isogeny class,  return the list of all labels of curves in that class, partitioned    into 2 parts where those in the first part are m-isogenous to E           with legendre(m,7)=+1 and those in the second part (which may be   empty) are m-isogenous to E with legendre(m,7)=-1.
 
 The items in the lists output are pairs (curve label, isogeny degree)

In [15]:
[all_c(c) for c in s]

[[('35378k1', 1), ('35378k2', 7)],
 [('176890be1', 1), ('176890be2', 7)],
 [('176890bg1', 1), ('176890bg2', 7)],
 [('389158bg1', 1), ('389158bg2', 7)]]

In [16]:
[[all_c(c) for c in s] for s in isom_sets_irred[:10]]

[[[('120150ci1', 1)], [('120150cp1', 1)]],
 [[('143220a1', 1)], [('221340d1', 1)]],
 [[('2412d1', 1)], [('335268d1', 1)]],
 [[('18538o1', 1)], [('53222f1', 1)]],
 [[('235200gi1', 1)], [('235200im1', 1), ('235200im2', 3)]],
 [[('79430j1', 1)], [('111202w1', 1)]],
 [[('18326z1', 1)], [('238238ba1', 1)]],
 [[('16560t1', 1)], [('281520fy1', 1)]],
 [[('57552k1', 1)], [('256368cx1', 1)]],
 [[('100800gw1', 1)], [('100800hc1', 1), ('100800hc2', 2)]]]

### Test one set, containing 2 isogeny classes:

In [17]:
isom_sets_irred[3]

['18538o1', '53222f1']

### The function test7iso() takes such a set of isogeny classes (of any size) and returns a list of length 2 lists of curve labels such that the curves in each sublist are sumplectically 7-isogenous.  The second list may be empty if all the isomorphisms are symplectic, as in this example:

In [18]:
test7iso(isom_sets_irred[3])

[['18538o1', '53222f1'], []]

### In this second example we see both syplectic and antisymplectic isomorphisms:

In [19]:
test7iso(isom_sets_irred[9])

[['100800gw1'], ['100800hc1', '100800hc2']]

### The preceding output means that '100800hc1' and '100800hc2' are symplectically isomorphic (in fact they are $2$-isogenous and $2$ is a quadratic non-residue) while '100800gw1' is anti-symplectically isomorphic to each.

In [20]:
all_c('100800hc1')

[('100800hc1', 1), ('100800hc2', 2)]

In [21]:
(test_isom_labels('100800hc1','100800hc2'),
test_isom_labels('100800gw1','100800hc1'),
test_isom_labels('100800gw1','100800hc2'))

(True, False, False)

## Here follows the long run which takes several hours

In [24]:
isom_sets_irred_symplectic2 = []
nsets = len(isom_sets_irred)
for i,s in enumerate(isom_sets_irred):
    print("Set #{}/{} ({:.3f}%)".format(i+1,nsets,float(100.0*(i+1)/nsets)))
    sys.stdout.write('\r')
    sys.stdout.flush()
    try:
        res = test7iso(s)
        isom_sets_irred_symplectic2.append(res)
        #print(res)
    except:
        print("error processing set number {}: {}".format(i,s))

Set #1/19883 (0.005%)
Set #2/19883 (0.010%)
Set #3/19883 (0.015%)
Set #4/19883 (0.020%)
Set #5/19883 (0.025%)
Set #6/19883 (0.030%)
Set #7/19883 (0.035%)
Set #8/19883 (0.040%)
Set #9/19883 (0.045%)
Set #10/19883 (0.050%)
Set #11/19883 (0.055%)
Set #12/19883 (0.060%)
Set #13/19883 (0.065%)
Set #14/19883 (0.070%)
Set #15/19883 (0.075%)
Set #16/19883 (0.080%)
Set #17/19883 (0.086%)
Set #18/19883 (0.091%)
Set #19/19883 (0.096%)
Set #20/19883 (0.101%)
Set #21/19883 (0.106%)
Set #22/19883 (0.111%)
Set #23/19883 (0.116%)
Set #24/19883 (0.121%)
Set #25/19883 (0.126%)
Set #26/19883 (0.131%)
Set #27/19883 (0.136%)
Set #28/19883 (0.141%)
Set #29/19883 (0.146%)
Set #30/19883 (0.151%)
Set #31/19883 (0.156%)
Set #32/19883 (0.161%)
Set #33/19883 (0.166%)
Set #34/19883 (0.171%)
Set #35/19883 (0.176%)
Set #36/19883 (0.181%)
Set #37/19883 (0.186%)
Set #38/19883 (0.191%)
Set #39/19883 (0.196%)
Set #40/19883 (0.201%)
Set #41/19883 (0.206%)
Set #42/19883 (0.211%)
Set #43/19883 (0.216%)
Set #44/19883 (0.221

Set #347/19883 (1.745%)
Set #348/19883 (1.750%)
Set #349/19883 (1.755%)
Set #350/19883 (1.760%)
Set #351/19883 (1.765%)
Set #352/19883 (1.770%)
Set #353/19883 (1.775%)
Set #354/19883 (1.780%)
Set #355/19883 (1.785%)
Set #356/19883 (1.790%)
Set #357/19883 (1.796%)
Set #358/19883 (1.801%)
Set #359/19883 (1.806%)
Set #360/19883 (1.811%)
Set #361/19883 (1.816%)
Set #362/19883 (1.821%)
Set #363/19883 (1.826%)
Set #364/19883 (1.831%)
Set #365/19883 (1.836%)
Set #366/19883 (1.841%)
Set #367/19883 (1.846%)
Set #368/19883 (1.851%)
Set #369/19883 (1.856%)
Set #370/19883 (1.861%)
Set #371/19883 (1.866%)
Set #372/19883 (1.871%)
Set #373/19883 (1.876%)
Set #374/19883 (1.881%)
Set #375/19883 (1.886%)
Set #376/19883 (1.891%)
Set #377/19883 (1.896%)
Set #378/19883 (1.901%)
Set #379/19883 (1.906%)
Set #380/19883 (1.911%)
Set #381/19883 (1.916%)
Set #382/19883 (1.921%)
Set #383/19883 (1.926%)
Set #384/19883 (1.931%)
Set #385/19883 (1.936%)
Set #386/19883 (1.941%)
Set #387/19883 (1.946%)
Set #388/19883 (

Set #689/19883 (3.465%)
Set #690/19883 (3.470%)
Set #691/19883 (3.475%)
Set #692/19883 (3.480%)
Set #693/19883 (3.485%)
Set #694/19883 (3.490%)
Set #695/19883 (3.495%)
Set #696/19883 (3.500%)
Set #697/19883 (3.506%)
Set #698/19883 (3.511%)
Set #699/19883 (3.516%)
Set #700/19883 (3.521%)
Set #701/19883 (3.526%)
Set #702/19883 (3.531%)
Set #703/19883 (3.536%)
Set #704/19883 (3.541%)
Set #705/19883 (3.546%)
Set #706/19883 (3.551%)
Set #707/19883 (3.556%)
Set #708/19883 (3.561%)
Set #709/19883 (3.566%)
Set #710/19883 (3.571%)
Set #711/19883 (3.576%)
Set #712/19883 (3.581%)
Set #713/19883 (3.586%)
Set #714/19883 (3.591%)
Set #715/19883 (3.596%)
Set #716/19883 (3.601%)
Set #717/19883 (3.606%)
Set #718/19883 (3.611%)
Set #719/19883 (3.616%)
Set #720/19883 (3.621%)
Set #721/19883 (3.626%)
Set #722/19883 (3.631%)
Set #723/19883 (3.636%)
Set #724/19883 (3.641%)
Set #725/19883 (3.646%)
Set #726/19883 (3.651%)
Set #727/19883 (3.656%)
Set #728/19883 (3.661%)
Set #729/19883 (3.666%)
Set #730/19883 (

Set #1030/19883 (5.180%)
Set #1031/19883 (5.185%)
Set #1032/19883 (5.190%)
Set #1033/19883 (5.195%)
Set #1034/19883 (5.200%)
Set #1035/19883 (5.205%)
Set #1036/19883 (5.210%)
Set #1037/19883 (5.216%)
Set #1038/19883 (5.221%)
Set #1039/19883 (5.226%)
Set #1040/19883 (5.231%)
Set #1041/19883 (5.236%)
Set #1042/19883 (5.241%)
Set #1043/19883 (5.246%)
Set #1044/19883 (5.251%)
Set #1045/19883 (5.256%)
Set #1046/19883 (5.261%)
Set #1047/19883 (5.266%)
Set #1048/19883 (5.271%)
Set #1049/19883 (5.276%)
Set #1050/19883 (5.281%)
Set #1051/19883 (5.286%)
Set #1052/19883 (5.291%)
Set #1053/19883 (5.296%)
Set #1054/19883 (5.301%)
Set #1055/19883 (5.306%)
Set #1056/19883 (5.311%)
Set #1057/19883 (5.316%)
Set #1058/19883 (5.321%)
Set #1059/19883 (5.326%)
Set #1060/19883 (5.331%)
Set #1061/19883 (5.336%)
Set #1062/19883 (5.341%)
Set #1063/19883 (5.346%)
Set #1064/19883 (5.351%)
Set #1065/19883 (5.356%)
Set #1066/19883 (5.361%)
Set #1067/19883 (5.366%)
Set #1068/19883 (5.371%)
Set #1069/19883 (5.376%)


Set #1358/19883 (6.830%)
Set #1359/19883 (6.835%)
Set #1360/19883 (6.840%)
Set #1361/19883 (6.845%)
Set #1362/19883 (6.850%)
Set #1363/19883 (6.855%)
Set #1364/19883 (6.860%)
Set #1365/19883 (6.865%)
Set #1366/19883 (6.870%)
Set #1367/19883 (6.875%)
Set #1368/19883 (6.880%)
Set #1369/19883 (6.885%)
Set #1370/19883 (6.890%)
Set #1371/19883 (6.895%)
Set #1372/19883 (6.900%)
Set #1373/19883 (6.905%)
Set #1374/19883 (6.910%)
Set #1375/19883 (6.915%)
Set #1376/19883 (6.920%)
Set #1377/19883 (6.926%)
Set #1378/19883 (6.931%)
Set #1379/19883 (6.936%)
Set #1380/19883 (6.941%)
Set #1381/19883 (6.946%)
Set #1382/19883 (6.951%)
Set #1383/19883 (6.956%)
Set #1384/19883 (6.961%)
Set #1385/19883 (6.966%)
Set #1386/19883 (6.971%)
Set #1387/19883 (6.976%)
Set #1388/19883 (6.981%)
Set #1389/19883 (6.986%)
Set #1390/19883 (6.991%)
Set #1391/19883 (6.996%)
Set #1392/19883 (7.001%)
Set #1393/19883 (7.006%)
Set #1394/19883 (7.011%)
Set #1395/19883 (7.016%)
Set #1396/19883 (7.021%)
Set #1397/19883 (7.026%)


...

Set #18428/19883 (92.682%)
Set #18429/19883 (92.687%)
Set #18430/19883 (92.692%)
Set #18431/19883 (92.697%)
Set #18432/19883 (92.702%)
Set #18433/19883 (92.707%)
Set #18434/19883 (92.712%)
Set #18435/19883 (92.717%)
Set #18436/19883 (92.722%)
Set #18437/19883 (92.727%)
Set #18438/19883 (92.732%)
Set #18439/19883 (92.738%)
Set #18440/19883 (92.743%)
Set #18441/19883 (92.748%)
Set #18442/19883 (92.753%)
Set #18443/19883 (92.758%)
Set #18444/19883 (92.763%)
Set #18445/19883 (92.768%)
Set #18446/19883 (92.773%)
Set #18447/19883 (92.778%)
Set #18448/19883 (92.783%)
Set #18449/19883 (92.788%)
Set #18450/19883 (92.793%)
Set #18451/19883 (92.798%)
Set #18452/19883 (92.803%)
Set #18453/19883 (92.808%)
Set #18454/19883 (92.813%)
Set #18455/19883 (92.818%)
Set #18456/19883 (92.823%)
Set #18457/19883 (92.828%)
Set #18458/19883 (92.833%)
Set #18459/19883 (92.838%)
Set #18460/19883 (92.843%)
Set #18461/19883 (92.848%)
Set #18462/19883 (92.853%)
Set #18463/19883 (92.858%)
Set #18464/19883 (92.863%)
S

Set #18732/19883 (94.211%)
Set #18733/19883 (94.216%)
Set #18734/19883 (94.221%)
Set #18735/19883 (94.226%)
Set #18736/19883 (94.231%)
Set #18737/19883 (94.236%)
Set #18738/19883 (94.241%)
Set #18739/19883 (94.246%)
Set #18740/19883 (94.251%)
Set #18741/19883 (94.256%)
Set #18742/19883 (94.261%)
Set #18743/19883 (94.266%)
Set #18744/19883 (94.271%)
Set #18745/19883 (94.277%)
Set #18746/19883 (94.282%)
Set #18747/19883 (94.287%)
Set #18748/19883 (94.292%)
Set #18749/19883 (94.297%)
Set #18750/19883 (94.302%)
Set #18751/19883 (94.307%)
Set #18752/19883 (94.312%)
Set #18753/19883 (94.317%)
Set #18754/19883 (94.322%)
Set #18755/19883 (94.327%)
Set #18756/19883 (94.332%)
Set #18757/19883 (94.337%)
Set #18758/19883 (94.342%)
Set #18759/19883 (94.347%)
Set #18760/19883 (94.352%)
Set #18761/19883 (94.357%)
Set #18762/19883 (94.362%)
Set #18763/19883 (94.367%)
Set #18764/19883 (94.372%)
Set #18765/19883 (94.377%)
Set #18766/19883 (94.382%)
Set #18767/19883 (94.387%)
Set #18768/19883 (94.392%)
S

Set #19036/19883 (95.740%)
Set #19037/19883 (95.745%)
Set #19038/19883 (95.750%)
Set #19039/19883 (95.755%)
Set #19040/19883 (95.760%)
Set #19041/19883 (95.765%)
Set #19042/19883 (95.770%)
Set #19043/19883 (95.775%)
Set #19044/19883 (95.780%)
Set #19045/19883 (95.785%)
Set #19046/19883 (95.790%)
Set #19047/19883 (95.795%)
Set #19048/19883 (95.800%)
Set #19049/19883 (95.805%)
Set #19050/19883 (95.810%)
Set #19051/19883 (95.816%)
Set #19052/19883 (95.821%)
Set #19053/19883 (95.826%)
Set #19054/19883 (95.831%)
Set #19055/19883 (95.836%)
Set #19056/19883 (95.841%)
Set #19057/19883 (95.846%)
Set #19058/19883 (95.851%)
Set #19059/19883 (95.856%)
Set #19060/19883 (95.861%)
Set #19061/19883 (95.866%)
Set #19062/19883 (95.871%)
Set #19063/19883 (95.876%)
Set #19064/19883 (95.881%)
Set #19065/19883 (95.886%)
Set #19066/19883 (95.891%)
Set #19067/19883 (95.896%)
Set #19068/19883 (95.901%)
Set #19069/19883 (95.906%)
Set #19070/19883 (95.911%)
Set #19071/19883 (95.916%)
Set #19072/19883 (95.921%)
S

Set #19340/19883 (97.269%)
Set #19341/19883 (97.274%)
Set #19342/19883 (97.279%)
Set #19343/19883 (97.284%)
Set #19344/19883 (97.289%)
Set #19345/19883 (97.294%)
Set #19346/19883 (97.299%)
Set #19347/19883 (97.304%)
Set #19348/19883 (97.309%)
Set #19349/19883 (97.314%)
Set #19350/19883 (97.319%)
Set #19351/19883 (97.324%)
Set #19352/19883 (97.329%)
Set #19353/19883 (97.334%)
Set #19354/19883 (97.339%)
Set #19355/19883 (97.344%)
Set #19356/19883 (97.349%)
Set #19357/19883 (97.355%)
Set #19358/19883 (97.360%)
Set #19359/19883 (97.365%)
Set #19360/19883 (97.370%)
Set #19361/19883 (97.375%)
Set #19362/19883 (97.380%)
Set #19363/19883 (97.385%)
Set #19364/19883 (97.390%)
Set #19365/19883 (97.395%)
Set #19366/19883 (97.400%)
Set #19367/19883 (97.405%)
Set #19368/19883 (97.410%)
Set #19369/19883 (97.415%)
Set #19370/19883 (97.420%)
Set #19371/19883 (97.425%)
Set #19372/19883 (97.430%)
Set #19373/19883 (97.435%)
Set #19374/19883 (97.440%)
Set #19375/19883 (97.445%)
Set #19376/19883 (97.450%)
S

Set #19644/19883 (98.798%)
Set #19645/19883 (98.803%)
Set #19646/19883 (98.808%)
Set #19647/19883 (98.813%)
Set #19648/19883 (98.818%)
Set #19649/19883 (98.823%)
Set #19650/19883 (98.828%)
Set #19651/19883 (98.833%)
Set #19652/19883 (98.838%)
Set #19653/19883 (98.843%)
Set #19654/19883 (98.848%)
Set #19655/19883 (98.853%)
Set #19656/19883 (98.858%)
Set #19657/19883 (98.863%)
Set #19658/19883 (98.868%)
Set #19659/19883 (98.873%)
Set #19660/19883 (98.878%)
Set #19661/19883 (98.883%)
Set #19662/19883 (98.888%)
Set #19663/19883 (98.894%)
Set #19664/19883 (98.899%)
Set #19665/19883 (98.904%)
Set #19666/19883 (98.909%)
Set #19667/19883 (98.914%)
Set #19668/19883 (98.919%)
Set #19669/19883 (98.924%)
Set #19670/19883 (98.929%)
Set #19671/19883 (98.934%)
Set #19672/19883 (98.939%)
Set #19673/19883 (98.944%)
Set #19674/19883 (98.949%)
Set #19675/19883 (98.954%)
Set #19676/19883 (98.959%)
Set #19677/19883 (98.964%)
Set #19678/19883 (98.969%)
Set #19679/19883 (98.974%)
Set #19680/19883 (98.979%)
S

In [22]:
[(i,s1,s2) for i,s1,s2 in zip(range(len(isom_sets_irred_symplectic2)), isom_sets_irred_symplectic, isom_sets_irred_symplectic2) if s1!=s2]

NameError: name 'isom_sets_irred_symplectic2' is not defined

In [25]:
len(isom_sets_irred_symplectic2)

19883

In [26]:
out = open("mod7irreducible_congruences.txt",'w')
for ss in isom_sets_irred_symplectic2:
    out.write(str(ss))
    out.write("\n")
out.close()    

In [27]:
isom_sets_irred_anti = [ss for ss in isom_sets_irred_symplectic2 if ss[1]]
len(isom_sets_irred_anti), len(isom_sets_irred_symplectic2)-len(isom_sets_irred_anti)

(7489, 12394)

In [25]:
from sage.databases.cremona import parse_cremona_label
parse_cremona_label("11a1")

(11, 'a', 1)

In [26]:
def class_label(label):
    return "".join([str(s) for s in parse_cremona_label(label)[:2]])

In [27]:
class_label("11a1")

'11a'

In [28]:
def non_triv(ss):
    classes = [[class_label(s) for s in si] for si in ss]
    return not any(c in classes[1] for c in classes[0])

In [32]:
isom_sets_irred_nontriv = [ss for ss in isom_sets_irred_anti if non_triv(ss)]
len(isom_sets_irred_nontriv)

5092

In [33]:
out = open("mod7_antipairs.m", "w")
out.write('pairs := [\\\n')
for s in isom_sets_irred_nontriv[:-1]:
    print(s[0],s[1])
    out.write('["{}", "{}"],\\\n'.format(s[0][0],s[1][0]))
s = isom_sets_irred_nontriv[-1]
out.write('["{}", "{}"]\\\n'.format(s[0][0],s[1][0]))
out.write('];\n')
out.close()

(['18326z1'], ['238238ba1'])
(['100800gw1'], ['100800hc1', '100800hc2'])
(['1178b1'], ['133114h1'])
(['1872r1', '1872r2', '1872r3', '1872r4'], ['87984bs1', '91728ft1'])
(['159936jh1'], ['159936jq1'])
(['204309p1'], ['364203v1'])
(['66240en1'], ['66240es1'])
(['9360bk1'], ['271440ch1'])
(['6534f1'], ['45738v1', '280962z1'])
(['17974i1'], ['17974j1'])
(['24780a1'], ['338660f1'])
(['28224et1'], ['28224gc1', '28224gc2'])
(['105105bb1'], ['105105bc1'])
(['87210w1'], ['87210x1'])
(['39984ct1'], ['39984do1'])
(['94640df1'], ['283920di1'])
(['3600br2', '3600br1'], ['46800ev1', '61200gs1'])
(['176400dm1'], ['176400fq1'])
(['95370dr1'], ['95370ds1'])
(['158025bh1'], ['158025bi1'])
(['144150bw1'], ['144150cn1'])
(['41292g1'], ['41292h1'])
(['19881g1'], ['139167i1'])
(['161994u1'], ['161994bg1'])
(['18928be1'], ['56784dc1'])
(['2704l1'], ['8112bf1'])
(['9675j1'], ['396675q1'])
(['3042a2', '3042a1'], ['57798d1'])
(['280476bd1'], ['280476be1'])
(['9702o1', '9702o2'], ['300762cn1'])
(['22050g1'], ['2

(['1950h1'], ['142350bp1'])
(['322896h1', '322896h2'], ['322896i2', '322896i1'])
(['122304gr1'], ['122304ij4', '122304ij2', '122304ij3', '122304ij1'])
(['10150d1'], ['131950ce1'])
(['198a4', '198a1', '198a2', '198a3'], ['4554o1', '110286j1'])
(['34496cd1'], ['103488gz1'])
(['3780c1'], ['223020bk1'])
(['275373l1'], ['275373m1'])
(['163350cx1'], ['163350ee1'])
(['310464ew1'], ['310464fb2', '310464fb1'])
(['16560bl1'], ['182160cr1'])
(['204490ch1'], ['204490cm1'])
(['35772a1'], ['276420f1'])
(['187425fk1'], ['253575fk1'])
(['17550cb1'], ['122850cr1'])
(['286650v1'], ['286650gk1'])
(['61446cc1'], ['102410bs1'])
(['318270p1'], ['318270q1'])
(['127050br1'], ['127050bz1'])
(['8820n2', '8820n1'], ['8820p1'])
(['101454c1'], ['150810l1'])
(['353934bg1', '353934bg2'], ['353934bh1'])
(['14514n1'], ['237062u1'])
(['78400a1'], ['78400s1'])
(['136710gm1'], ['136710gr1'])
(['302400lt1'], ['302400sa1'])
(['17424br1'], ['87120el1'])
(['132940b1'], ['398820bn1'])
(['168432i1'], ['168432j1'])
(['141120iz1

(['91728fs1'], ['91728fw2', '91728fw1'])
(['2800bf1'], ['8400bw1', '8400bw2'])
(['152460x1'], ['152460z1', '152460z2'])
(['344988f1'], ['344988h1'])
(['134946cu1'], ['134946cx1'])
(['1360e2', '1360e1'], ['131920l1'])
(['11466g1', '11466g2'], ['217854en1'])
(['368f1', '54096bn1'], ['18768o1'])
(['2160u1', '153360p1'], ['41040br1'])
(['34914a1'], ['174570da1'])
(['19008br1'], ['95040gw1'])
(['3612g1'], ['299796i1'])
(['6084p1', '6084p2'], ['298116s1'])
(['20286c1'], ['385434ck1'])
(['225302cx1'], ['225302cy1'])
(['154154m1'], ['154154ba1'])
(['4554bh1'], ['159390e2', '159390e1'])
(['282150cx1'], ['282150fe1'])
(['52800fy1'], ['123200ga1'])
(['35490cf1', '35490cf2'], ['291018ch1'])
(['31050bi1'], ['341550eb1'])
(['37926g1', '37926p1'], ['37926r1'])
(['63296o1', '217408e1'], ['244928d1'])
(['2975a1'], ['235025a1'])
(['36300bl1'], ['36300bm1'])
(['170352c1'], ['170352cu1'])
(['282576k3', '282576k2', '282576k1', '282576k6', '282576k5', '282576k4'], ['282576o1'])
(['1650e1', '51150t1'], ['169

In [34]:
out = open("mod7_antipairs_all.m", "w")
out.write('pairs := [\\\n')
for s in isom_sets_irred_symplectic2:
    for s0 in s[0]:
        for s1 in s[1]:
            out.write('["{}", "{}"],\\\n'.format(s0,s1))
#s = isom_sets_irred_symplectic2[-1]
#out.write('["{}", "{}"]\\\n'.format(s[0][0],s[1][0]))
out.write('];\n')
out.close()

In [15]:
%run XE7

In [16]:
R2.<t> = QQ[]

In [104]:
j0sets = [s for s in isom_sets_irred if any([EllipticCurve(lab).j_invariant()==0 for lab in s])]

In [248]:
asq_sets = [isom_sets_irred[i] for i in [22,27,43,63,70,71,76,84]]
for s in asq_sets:
    print(test7iso(s))

[['9016m1', '171304h1', '333592j1'], []]
[['5550bk1', '5550bm4', '5550bm1', '5550bm3', '5550bm2'], []]
[['33856q1', '169280bl1'], []]
[['9800f1', '9800g1'], []]
[['82810b1', '82810g1'], []]
[['40768a1', '40768p1'], []]
[['174048a1', '174048p1'], []]
[['290325bt1', '290325bu1'], []]


## Testing isomorphisms among sets isogeny classes which are reducible and mutually isomorphic up to semisimplification

In [22]:
len(isom_sets_red)

255

In [23]:
%run testcong.py

In [24]:
isom_sets_red_complete = []
for i, ss in enumerate(isom_sets_red):
    S1, S2 = mod_p_iso_red_set(ss,7,0)
    #print(i, S1, S2)
    nontriv = [s for s in S1+S2 if len(s)>1]
    if nontriv:
        print(nontriv)
        isom_sets_red_complete += nontriv
        

[['146205i2', '321651n2']]
[['2704d1', '13520o1'], ['2704d2', '100048e2']]
[['28224fo2', '141120bs2'], ['7488bz1', '52416fg1', '82368ea1'], ['118080fv1', '165312z1'], ['28224fo1', '65088dg2']]
[['78400dk2', '235200pp2']]
[['155682f2', '155682i3']]
[['10816bk1', '227136t1', '356928fw1']]
[['1296k1', '138672f1'], ['1296k3', '40176r2']]
[['161728i1', '254144cd1']]
[['161728j1', '254144cc1']]
[['78400p2', '235200x2']]
[['12675v1', '392925cf1'], ['12675v2', '257725i2']]
[['35574d2', '59290f2']]
[['11025y1', '143325ev2'], ['11025y3', '143325eq2']]
[['25270p1', '299630q1'], ['5054b1', '7942p1'], ['3610f2', '107578j2']]
[['368082bk1', '368082bn1']]
[['25168bp1', '75504bt1']]
[['17298e2', '294066h2']]
[['5184a3', '160704bd2']]
[['8256c1', '151360da1'], ['9408h2', '15680ba2'], ['832f1', '17472m1', '27456g1'], ['18368j1', '39360b1'], ['9408h1', '21696h2']]
[['8112bd2', '164944b2']]
[['74529w2', '74529x3'], ['74529m2', '74529x1']]
[['507a1', '78585g1'], ['16055e2', '124215bc1'], ['507a2', '10309b2

[['99225bf1', '99225bh2']]
[['3042b1', '15210l1'], ['3042b2', '112554f2']]
[['891d1', '223641h1'], ['405c1', '6885c1', '284715m1'], ['405c2', '891d2', '3969c1']]
[['2025c1', '34425d1'], ['2025c2', '22275f2', '99225l1']]
[['16562d1', '82810bc1']]
[['40768bc1', '122304dd1'], ['9408v2', '15680bl2']]
[['12675y1', '392925v1'], ['12675y2', '257725a2']]
[['891g1', '223641c1'], ['405d1', '6885h1', '284715e1'], ['405d2', '891g2', '3969f1']]
[['25920cw2', '57024cx2', '254016do1']]
[['40898bg1', '204490ba1']]
[['4050bh3', '125550dh2']]
[['1225b1', '15925d2', '221725f1'], ['1225b3', '15925h2', '221725e2']]
[['226512ce1', '226512ch1']]
[['100672y1', '302016cp1']]
[['245025r2', '245025v2']]
[['22050eb2', '22050ev2'], ['19350by1', '212850v1'], ['5850bq1', '40950ea1', '64350du1', '286650ou2'], ['18450bu1', '129150cr1'], ['22050ev1', '50850bd2']]
[['76050et1', '76050fc1']]
[['165886k1', '355470bj1'], ['7514i1', '157794bc1', '247962y1'], ['84966ct2', '141610bk2'], ['84966ct1', '195942p2']]
[['366912ju1'

In [25]:
len(isom_sets_red_complete)

337

In [26]:
[(n, len([s for s in isom_sets_red_complete if len(s)==n])) for n in [2,3,4]]

[(2, 276), (3, 52), (4, 9)]

In [27]:
Set(len(s) for s in isom_sets_red_complete)

{2, 3, 4}

In [28]:
isom_sets_red_split = []
for i,ss in enumerate(isom_sets_red_complete):
    E1 = ss[0]
    symps = [E1]
    antis = []
    for E2 in ss[1:]:
        res = test_isom_labels(E1,E2, False)
        print("The isomorphism between {} and {} is {}".format(E1,E2,"symplectic" if res else "antisyplectic"))
        (symps if res else antis).append(E2)
    isom_sets_red_split.append([symps,antis])    
    print("{}: {}={}+{}".format(i,len(ss),len(symps),len(antis)))

The isomorphism between 146205i2 and 321651n2 is symplectic
0: 2=2+0
The isomorphism between 2704d1 and 13520o1 is antisyplectic
1: 2=1+1
The isomorphism between 2704d2 and 100048e2 is antisyplectic
2: 2=1+1
The isomorphism between 28224fo2 and 141120bs2 is antisyplectic
3: 2=1+1
The isomorphism between 7488bz1 and 52416fg1 is symplectic
The isomorphism between 7488bz1 and 82368ea1 is antisyplectic
4: 3=2+1
The isomorphism between 118080fv1 and 165312z1 is symplectic
5: 2=2+0
The isomorphism between 28224fo1 and 65088dg2 is symplectic
6: 2=2+0
The isomorphism between 78400dk2 and 235200pp2 is antisyplectic
7: 2=1+1
The isomorphism between 155682f2 and 155682i3 is antisyplectic
8: 2=1+1
The isomorphism between 10816bk1 and 227136t1 is symplectic
The isomorphism between 10816bk1 and 356928fw1 is antisyplectic
9: 3=2+1
The isomorphism between 1296k1 and 138672f1 is antisyplectic
10: 2=1+1
The isomorphism between 1296k3 and 40176r2 is symplectic
11: 2=2+0
The isomorphism between 161728i1 a

The isomorphism between 162c3 and 5022d2 is symplectic
105: 2=2+0
The isomorphism between 167994i2 and 184518l2 is symplectic
106: 2=2+0
The isomorphism between 162b1 and 17334h1 is symplectic
The isomorphism between 162b1 and 293706ci2 is antisyplectic
107: 3=2+1
The isomorphism between 162b3 and 5022g2 is antisyplectic
108: 2=1+1
The isomorphism between 167994c2 and 184518g2 is symplectic
109: 2=2+0
The isomorphism between 28224x2 and 141120ng2 is antisyplectic
110: 2=1+1
The isomorphism between 366912kd1 and 366912lm1 is symplectic
111: 2=2+0
The isomorphism between 7488x1 and 52416cz1 is symplectic
The isomorphism between 7488x1 and 82368ce1 is antisyplectic
112: 3=2+1
The isomorphism between 118080cm1 and 165312eg1 is symplectic
113: 2=2+0
The isomorphism between 28224bs2 and 141120it2 is antisyplectic
114: 2=1+1
The isomorphism between 28224bs1 and 65088ba2 is symplectic
115: 2=2+0
The isomorphism between 5184u3 and 160704m2 is antisyplectic
116: 2=1+1
The isomorphism between 313

The isomorphism between 7350j1 and 16950e2 is symplectic
203: 2=2+0
The isomorphism between 161728bk1 and 254144ce1 is antisyplectic
204: 2=1+1
The isomorphism between 49a1 and 637a2 is symplectic
The isomorphism between 49a1 and 8869d1 is symplectic
205: 3=3+0
The isomorphism between 27391f1 and 61789g1 is symplectic
The isomorphism between 27391f1 and 88543g1 is symplectic
206: 3=3+0
The isomorphism between 61103g1 and 117943b2 is symplectic
207: 2=2+0
The isomorphism between 61103a1 and 117943f2 is symplectic
208: 2=2+0
The isomorphism between 27391b2 and 61789d2 is symplectic
The isomorphism between 27391b2 and 88543d2 is symplectic
209: 3=3+0
The isomorphism between 49a3 and 637c2 is symplectic
The isomorphism between 49a3 and 8869i2 is symplectic
210: 3=3+0
The isomorphism between 63504y1 and 317520hf2 is symplectic
211: 2=2+0
The isomorphism between 8450t1 and 177450di1 is symplectic
The isomorphism between 8450t1 and 278850gb1 is antisyplectic
212: 3=2+1
The isomorphism between

The isomorphism between 405c1 and 6885c1 is symplectic
The isomorphism between 405c1 and 284715m1 is symplectic
302: 3=3+0
The isomorphism between 405c2 and 891d2 is symplectic
The isomorphism between 405c2 and 3969c1 is symplectic
303: 3=3+0
The isomorphism between 2025c1 and 34425d1 is symplectic
304: 2=2+0
The isomorphism between 2025c2 and 22275f2 is symplectic
The isomorphism between 2025c2 and 99225l1 is symplectic
305: 3=3+0
The isomorphism between 16562d1 and 82810bc1 is antisyplectic
306: 2=1+1
The isomorphism between 40768bc1 and 122304dd1 is symplectic
307: 2=2+0
The isomorphism between 9408v2 and 15680bl2 is antisyplectic
308: 2=1+1
The isomorphism between 12675y1 and 392925v1 is symplectic
309: 2=2+0
The isomorphism between 12675y2 and 257725a2 is symplectic
310: 2=2+0
The isomorphism between 891g1 and 223641c1 is symplectic
311: 2=2+0
The isomorphism between 405d1 and 6885h1 is symplectic
The isomorphism between 405d1 and 284715e1 is symplectic
312: 3=3+0
The isomorphism 

In [29]:
assert len(isom_sets_red_split)==len(isom_sets_red_complete)

In [30]:
set([tuple([len(s) for s in ss]) for ss in isom_sets_red_split])

{(1, 1), (1, 2), (2, 0), (2, 1), (3, 0), (3, 1)}

In [31]:
ofile_s = open("mod7_red_symp_pairs.m", 'w')
ofile_a = open("mod7_red_anti_pairs.m", 'w')
ofile_s.write('pairs := [\\\n')
ofile_a.write('pairs := [\\\n')
first_a = first_s = True
for ss in isom_sets_red_split:
    for s in ss:
        if s:
            s0 = s[0]
            for s1 in s[1:]:
                if not first_s:
                    ofile_s.write(", ")
                else:
                    first_s = False
                ofile_s.write('["{}", "{}"]\\\n'.format(s0,s1))
    s0 = ss[0][0]
    for s1 in ss[1]:
        if not first_a:
            ofile_a.write(", ")
        else:
            first_a = False
        ofile_a.write('["{}", "{}"]\\\n'.format(s0,s1))
ofile_s.write('];\n')
ofile_a.write('];\n')
ofile_s.close()
ofile_a.close()

In [44]:
t = polygen(QQ,'t')
E0 = KH0(t)
E1 = KH1(t)
E2 = KH2(t)

In [45]:
E0.discriminant().factor()

(-338688) * t * (t + 49/27)

In [36]:
E1.discriminant().factor()

(-432) * (t + 49/27) * t^2 * (t + 3)^7

In [37]:
E2.discriminant().factor()

(-122009559759792) * t * (t + 49/27)^2 * (t - 32/27)^7

In [43]:
E1.discriminant().gcd(E1.c6())

1

In [72]:
Ea0 = KH0(-3)
Ea0.label(), Ea0.j_invariant()

('294b1', -2401/6)

In [65]:
Ea2 = KH2(-3)
Ea2.label(), Ea2.j_invariant()

('678d2', -39934705050538129/2823126576537804)

In [66]:
Eb0 = KH0(32/27)
Eb0.label(), Eb0.j_invariant()

('2352l1', -2401/6)

In [67]:
Eb1 = KH1(32/27)
Eb1.label(), Eb1.j_invariant()

('5424l2', -39934705050538129/2823126576537804)

In [68]:
Eb0.is_quadratic_twist(Ea0)

-1

In [70]:
E0.j_invariant().factor()

(-38416/27) * t^-1 * (t + 49/27)^-1

In [76]:
KH0(-(49/27)-t).is_quadratic_twist(KH0(t))

AttributeError: 'EllipticCurve_generic_with_category' object has no attribute 'is_quadratic_twist'